# Librerias utilizadas

In [ ]:
import os
import re
import json
import time
import random
import subprocess
import smtplib
import platform
import requests
import joblib
import unicodedata

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from email.message import EmailMessage
from collections import defaultdict
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from rich import print
from rich.console import Console
from rich.table import Table
from rich.progress import track

import undetected_chromedriver as uc

import openai

import warnings
warnings.simplefilter("ignore")


# Datos

In [ ]:
# 📌 Email configuration
SMTP_SERVER =  "..."
SMTP_PORT = ...
EMAIL_SENDER =  "..."
EMAIL_PASSWORD =  "..."


## 1. Scraping

In [ ]:
REALIZAR_SCRAPING = True  # Change to False to avoid scraping

console = Console()

# Detect the operating system and set the appropriate User-Agent
system = platform.system()
if system == "Windows":
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
elif system == "Darwin":  # macOS
    user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
else:  # Linux or others
    user_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"

# Function to start a new browser
def iniciar_navegador():
    options = uc.ChromeOptions()
    options.add_argument(f"user-agent={user_agent}")
    options.headless = False  # or True if you don’t want the browser to be visible
    # 🔧 Key line: prevents zombie process errors
    return uc.Chrome(options=options, use_subprocess=True)

# Function to safely restart the browser
def reiniciar_navegador():
    global browser
    try:
        browser.quit()
    except Exception as e:
        console.print(f"[yellow]⚠️ Could not close the browser normally:[/yellow] {e}")
    time.sleep(5)  # Allow time to close properly
    browser = iniciar_navegador()

# Start browser
browser = iniciar_navegador()

# CSV file paths
ids_csv_path = '../data/ids_casas.csv'
casas_csv_path = '../data/casas_idealista.csv'

# Load existing IDs
ids_existentes = set()
try:
    ids_existentes = set(pd.read_csv(ids_csv_path)['id'].dropna().astype(str).tolist())
except FileNotFoundError:
    pass

pagina = 1
nuevos_ids = []
repetidos_consecutivos = 0
limite_repetidos = 5

try:
    while REALIZAR_SCRAPING:
        url = f'https://www.idealista.com/venta-viviendas/zaragoza-zaragoza/pagina-{pagina}.htm?ordenado-por=fecha-publicacion-desc'
        console.print(f"\n[bold cyan]🌍 Accessing page:[/bold cyan] {pagina}")

        try:
            browser.get(url)
            browser.delete_all_cookies()
            time.sleep(random.uniform(8, 15))
        except Exception as e:
            console.print(f"[bold red]⚠️ Error accessing page {pagina}:[/bold red] {e}")
            reiniciar_navegador()
            continue

        # Try to close the cookies notice
        try:
            browser.find_element("xpath", '//*[@id="didomi-notice-agree-button"]').click()
        except:
            pass

        soup = bs(browser.page_source, 'lxml')
        articles = soup.find('main', {'class': 'listing-items'}).find_all('article')

        console.print(f"[bold green]🏡 Number of properties found:[/bold green] {len(articles)}")

        for article in track(articles, description="📥 Processing properties..."):
            id_muebles = article.get('data-element-id')

            if id_muebles:
                if str(id_muebles) in ids_existentes:
                    repetidos_consecutivos += 1
                    console.print(f"🔁 [yellow]Duplicate ID found:[/yellow] {id_muebles} (Total consecutive duplicates: {repetidos_consecutivos})")

                    if repetidos_consecutivos >= limite_repetidos:
                        raise StopIteration  # Stop if too many consecutive duplicates

                    continue

                repetidos_consecutivos = 0
                nuevos_ids.append(id_muebles)

        pagina += 1
        time.sleep(random.uniform(5, 10))

except StopIteration:
    console.print("\n[bold red]🚨 Scraping stopped due to multiple consecutive duplicates.[/bold red]")

except KeyboardInterrupt:
    console.print("\n[bold red]🛑 Scraping manually interrupted.[/bold red]")

finally:
    df_nuevas_casas = pd.DataFrame()
    i = 0

    while i < len(nuevos_ids):
        nuevo_id = nuevos_ids[i]
        console.print(f"📌 [cyan]Processing data for ID:[/cyan] {nuevo_id}")

        try:
            url = f"https://www.idealista.com/inmueble/{nuevo_id}/"

            intentos = 0
            max_intentos = 2
            titulo = None

            while intentos < max_intentos and not titulo:
                try:
                    browser.get(url)
                    time.sleep(random.uniform(4, 6))
                    html = browser.page_source
                    soup = bs(html, 'lxml')

                    # Look for title to confirm the page loaded correctly
                    titulo = soup.find('span', {'class': 'main-info__title-main'})

                    if not titulo:
                        raise Exception("Property title not found.")

                except Exception as e:
                    intentos += 1
                    console.print(f"🔁 [yellow]Attempt {intentos}/{max_intentos} failed for property {nuevo_id}:[/yellow] {e}")

                    if intentos == 1:
                        console.print("[bold yellow]🛑 Enable VPN or change IP, then press Enter to continue...[/bold yellow]")
                        input("▶️ Press Enter when ready: ")
                        reiniciar_navegador()

            if not titulo:
                console.print(f"⚠️ [red]Property {nuevo_id} not available. It may have been removed or you are still blocked.[/red]")
                i += 1
                continue

            titulo = soup.find('span', {'class': 'main-info__title-main'})
            localizacion = soup.find('span', {'class': 'main-info__title-minor'})
            precio_tag = soup.find('span', {'class': 'txt-bold'})
            c1 = soup.find('div', {'class': 'details-property-feature-one'})
            c2 = soup.find('div', {'class': 'details-property-feature-two'})

            casas = {
                'id': nuevo_id,
                'Titulo': titulo.text if titulo else None,
                'Localizacion': localizacion.text.split(',')[0] if localizacion else None,
                'Precio': int(precio_tag.text.replace('.', '').replace('€', '').strip()) if precio_tag else None,
                'Caracteristicas_basicas': '; '.join([caract.text.strip() for caract in c1.find_all('li')]) if c1 else None,
                'Caracteristicas_extra': '; '.join([caract.text.strip() for caract in c2.find_all('li')]) if c2 else None
            }

            df_casas = pd.DataFrame([casas])
            df_nuevas_casas = pd.concat([df_nuevas_casas, df_casas], ignore_index=True)
            i += 1  # Only move forward if successful

        except Exception as e:
            console.print(f"❌ [red]Error processing property {nuevo_id}:[/red] {e}")
            i += 1  # Or use `continue` if you want to retry later

    browser.quit()

    if not df_nuevas_casas.empty:
        df_nuevas_casas.to_csv(casas_csv_path, index=False, sep=';', encoding='utf-16')
        console.print(f"\n💾 [green]Data for {len(df_nuevas_casas)} properties saved to:[/green] {casas_csv_path}")

        try:
            df_ids_nuevos = pd.DataFrame(df_nuevas_casas['id'], columns=['id'])
            df_ids_nuevos.to_csv(ids_csv_path, mode='a', header=False, index=False)
            console.print(f"✅ [bold green]New IDs added to:[/bold green] {ids_csv_path} ({len(df_nuevas_casas)} IDs)")
        except Exception as e:
            console.print(f"❌ [red]Error saving new IDs to {ids_csv_path}:[/red] {e}")
    else:
        console.print("⚠️ [yellow]No data found to save.[/yellow]")


### 1.1 Enmaquetado

In [ ]:
# 📂 Load CSV file ignoring errors
console.print("[cyan]📂 Cargando datos desde 'casas_idealista.csv'...[/cyan]")
try:
    df = pd.read_csv('../data/casas_idealista.csv', encoding='utf-16', sep=';')
    console.print(f"✅ [green]Archivo cargado correctamente con {len(df)} registros.[/green]")
except Exception as e:
    console.print(f"❌ [red]Error al cargar el archivo:[/red] {e}")
    df = pd.DataFrame()  # Create an empty DataFrame

# Drop 'Titulo' column
df = df.drop(columns=['Titulo'], errors='ignore')

# Drop rows with NaN in any column
df_limpio = df.dropna()

console.print(f"🔍 [yellow]Registros después de eliminar filas con NaN:[/yellow] {len(df_limpio)}")

# Function to extract square meters
def extraer_m2(texto):
    match_construidos = re.search(r'(\d+)\s*m² construidos', texto)
    match_utiles = re.search(r'(\d+)\s*m² útiles', texto)
    return int(match_construidos.group(1)) if match_construidos else None, \
           int(match_utiles.group(1)) if match_utiles else None

# Apply extraction of square meters
df[['m2_construidos', 'm2_utiles']] = df['Caracteristicas_basicas'].apply(lambda x: pd.Series(extraer_m2(str(x))))

console.print("🏠 [cyan]Metros cuadrados extraídos correctamente.[/cyan]")

# Extract bedrooms and bathrooms
def extraer_habitaciones_banos(texto):
    match_habitaciones = re.search(r'(\d+)\s*habitaci[oó]n(?:es)?', texto)
    match_banos = re.search(r'(\d+)\s*bañ(?:o|os)', texto)
    return int(match_habitaciones.group(1)) if match_habitaciones else None, \
           int(match_banos.group(1)) if match_banos else None

df[['habitaciones', 'banos']] = df['Caracteristicas_basicas'].apply(lambda x: pd.Series(extraer_habitaciones_banos(str(x))))

console.print("🛏️ [cyan]Habitaciones y baños extraídos correctamente.[/cyan]")

# Extract property condition
def extraer_estado_vivienda(texto):
    texto = texto.lower()
    if "segunda mano/buen estado" in texto:
        return "Segunda mano/Buen estado"
    elif "segunda mano/para reformar" in texto:
        return "Segunda mano/Para reformar"
    elif "promoción de obra nueva" in texto:
        return "Obra nueva"
    return None

# Detect if it is exterior or interior as binary variables
df['Exterior'] = df['Caracteristicas_basicas'].apply(
    lambda x: 1 if isinstance(x, str) and 'exterior' in x.lower() else 0
)

df['Interior'] = df['Caracteristicas_basicas'].apply(
    lambda x: 1 if isinstance(x, str) and 'interior' in x.lower() else 0
)

df['estado_vivienda'] = df['Caracteristicas_basicas'].apply(lambda x: extraer_estado_vivienda(str(x)))

console.print("🏚️ [cyan]Estado de la vivienda extraído correctamente.[/cyan]")

# Create binary variables for features
caracteristicas = {
    "trastero": "Trastero",
    "terraza": "Terraza",
    "balcon": "Balcón",
    "ascensor": "con ascensor",
    "armarios_empotrados": "armarios empotrados",
    "plaza_garaje": "garaje",
    "garaje_incluido": "incluida",
    "garaje_adicional": "adicionales",
    "adaptado_movilidad_reducida": "movilidad reducida"
}

for col, keyword in caracteristicas.items():
    df[col] = df['Caracteristicas_basicas'].apply(lambda x: 1 if isinstance(x, str) and keyword.lower() in x.lower() else 0)

console.print("✅ [green]Variables binarias extraídas correctamente.[/green]")

# Extract orientations
def detectar_orientaciones(texto):
    orientaciones = {'orientacion_este': 0, 'orientacion_oeste': 0, 'orientacion_norte': 0, 'orientacion_sur': 0}
    if pd.isna(texto):
        return orientaciones
    texto = texto.lower()
    for orientacion in orientaciones.keys():
        if orientacion.replace('orientacion_', '') in texto:
            orientaciones[orientacion] = 1
    return orientaciones

orientaciones_df = df['Caracteristicas_basicas'].apply(detectar_orientaciones).apply(pd.Series)
df = pd.concat([df, orientaciones_df], axis=1)

console.print("🧭 [cyan]Orientaciones extraídas correctamente.[/cyan]")

# Extract heating
df['Calefacción'] = df['Caracteristicas_basicas'].str.extract(r'Calefacción ([^;]+)').fillna('sin calefacción').astype('category')

# Extract year of construction
def extraer_ano_construccion(texto):
    match = re.search(r'construido en\s*(\d+)', texto.lower())
    return int(match.group(1)) if match else None

df['ano_construccion'] = df['Caracteristicas_basicas'].apply(lambda x: extraer_ano_construccion(str(x))).astype('Int64')

# Use regex to extract the number following the word "Planta"
df['planta_numero'] = df['Caracteristicas_basicas'].str.extract(r'Planta (\d+)')

# Convert the new column to a numeric type (integer)
df['planta_numero'] = pd.to_numeric(df['planta_numero'], errors='coerce').astype('Int64')

console.print("🏗️ [cyan]Año de construcción extraído correctamente.[/cyan]")

# Function to detect extra features
def extraer_caracteristicas_extras(texto):
    # Convert text to lowercase for standardization
    texto = texto.lower()
    
    # Detect presence of each feature
    jardin = 1 if 'jardín' in texto else 0
    piscina = 1 if 'piscina' in texto else 0
    aire_acondicionado = 1 if 'aire acondicionado' in texto else 0
    zonas_verdes = 1 if 'zonas verdes' in texto else 0
    
    return pd.Series([jardin, piscina, aire_acondicionado, zonas_verdes])

# Apply function to 'Caracteristicas_extra' and create new columns
df[['jardin', 'piscina', 'aire_acondicionado', 'zonas_verdes']] = df['Caracteristicas_extra'].apply(lambda x: extraer_caracteristicas_extras(str(x)))

# Drop original feature columns
df = df.drop(columns=['Caracteristicas_basicas', 'Caracteristicas_extra'], errors='ignore')

# Save processed DataFrame
output_path = '../data/casas_idealista_procesado_extra.csv'
df.to_csv(output_path, index=False, sep=';', encoding='utf-8')

console.print(f"💾 [green]Datos guardados en '{output_path}'.[/green]")

# Load files to combine
csv_file_1 = '../data/casas_idealista_procesado.csv'
csv_file_2 = '../data/casas_idealista_procesado_extra.csv'
output_csv_file = '../data/casas_idealista_procesado.csv'

def cargar_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8', sep=';')
        return df
    except FileNotFoundError:
        console.print(f"⚠️ [yellow]Archivo no encontrado: {file_path}[/yellow]")
        return pd.DataFrame()
    except Exception as e:
        console.print(f"❌ [red]Error al leer {file_path}: {e}[/red]")
        return pd.DataFrame()

df1 = cargar_csv(csv_file_1)
df2 = cargar_csv(csv_file_2)

# Combine DataFrames
if not df1.empty and not df2.empty:
    df_combined = pd.concat([df1, df2], ignore_index=True)
    console.print(f"📊 [cyan]Datos combinados: {len(df_combined)} registros.[/cyan]")
else:
    if df1.empty:
        console.print(f"⚠️ [yellow]El archivo '{csv_file_1}' está vacío.[/yellow]")
    if df2.empty:
        console.print(f"⚠️ [yellow]El archivo '{csv_file_2}' está vacío.[/yellow]")

# Save combined result
if not df_combined.empty:
    df_combined.to_csv(output_csv_file, index=False, sep=';', encoding='utf-8')
    console.print(f"✅ [green]Datos guardados en '{output_csv_file}'.[/green]")
else:
    console.print("❌ [red]No hay datos para guardar en el archivo final.[/red]")


# 2. Machine Learning

In [ ]:
# 📂 Load CSV file ignoring errors
console.print("[cyan]📂 Cargando datos desde 'casas_idealista.csv'...[/cyan]")
try:
    df = pd.read_csv('../data/casas_idealista.csv', encoding='utf-16', sep=';')
    console.print(f"✅ [green]Archivo cargado correctamente con {len(df)} registros.[/green]")
except Exception as e:
    console.print(f"❌ [red]Error al cargar el archivo:[/red] {e}")
    df = pd.DataFrame()  # Create an empty DataFrame

# Drop 'Titulo' column
df = df.drop(columns=['Titulo'], errors='ignore')

# Drop rows with NaN in any column
df_limpio = df.dropna()

console.print(f"🔍 [yellow]Registros después de eliminar filas con NaN:[/yellow] {len(df_limpio)}")

# Function to extract square meters
def extraer_m2(texto):
    match_construidos = re.search(r'(\d+)\s*m² construidos', texto)
    match_utiles = re.search(r'(\d+)\s*m² útiles', texto)
    return int(match_construidos.group(1)) if match_construidos else None, \
           int(match_utiles.group(1)) if match_utiles else None

# Apply extraction of square meters
df[['m2_construidos', 'm2_utiles']] = df['Caracteristicas_basicas'].apply(lambda x: pd.Series(extraer_m2(str(x))))

console.print("🏠 [cyan]Metros cuadrados extraídos correctamente.[/cyan]")

# Extract bedrooms and bathrooms
def extraer_habitaciones_banos(texto):
    match_habitaciones = re.search(r'(\d+)\s*habitaci[oó]n(?:es)?', texto)
    match_banos = re.search(r'(\d+)\s*bañ(?:o|os)', texto)
    return int(match_habitaciones.group(1)) if match_habitaciones else None, \
           int(match_banos.group(1)) if match_banos else None

df[['habitaciones', 'banos']] = df['Caracteristicas_basicas'].apply(lambda x: pd.Series(extraer_habitaciones_banos(str(x))))

console.print("🛏️ [cyan]Habitaciones y baños extraídos correctamente.[/cyan]")

# Extract property condition
def extraer_estado_vivienda(texto):
    texto = texto.lower()
    if "segunda mano/buen estado" in texto:
        return "Segunda mano/Buen estado"
    elif "segunda mano/para reformar" in texto:
        return "Segunda mano/Para reformar"
    elif "promoción de obra nueva" in texto:
        return "Obra nueva"
    return None

# Detect if it is exterior or interior as binary variables
df['Exterior'] = df['Caracteristicas_basicas'].apply(
    lambda x: 1 if isinstance(x, str) and 'exterior' in x.lower() else 0
)

df['Interior'] = df['Caracteristicas_basicas'].apply(
    lambda x: 1 if isinstance(x, str) and 'interior' in x.lower() else 0
)

df['estado_vivienda'] = df['Caracteristicas_basicas'].apply(lambda x: extraer_estado_vivienda(str(x)))

console.print("🏚️ [cyan]Estado de la vivienda extraído correctamente.[/cyan]")

# Create binary variables for features
caracteristicas = {
    "trastero": "Trastero",
    "terraza": "Terraza",
    "balcon": "Balcón",
    "ascensor": "con ascensor",
    "armarios_empotrados": "armarios empotrados",
    "plaza_garaje": "garaje",
    "garaje_incluido": "incluida",
    "garaje_adicional": "adicionales",
    "adaptado_movilidad_reducida": "movilidad reducida"
}

for col, keyword in caracteristicas.items():
    df[col] = df['Caracteristicas_basicas'].apply(lambda x: 1 if isinstance(x, str) and keyword.lower() in x.lower() else 0)

console.print("✅ [green]Variables binarias extraídas correctamente.[/green]")

# Extract orientations
def detectar_orientaciones(texto):
    orientaciones = {'orientacion_este': 0, 'orientacion_oeste': 0, 'orientacion_norte': 0, 'orientacion_sur': 0}
    if pd.isna(texto):
        return orientaciones
    texto = texto.lower()
    for orientacion in orientaciones.keys():
        if orientacion.replace('orientacion_', '') in texto:
            orientaciones[orientacion] = 1
    return orientaciones

orientaciones_df = df['Caracteristicas_basicas'].apply(detectar_orientaciones).apply(pd.Series)
df = pd.concat([df, orientaciones_df], axis=1)

console.print("🧭 [cyan]Orientaciones extraídas correctamente.[/cyan]")

# Extract heating
df['Calefacción'] = df['Caracteristicas_basicas'].str.extract(r'Calefacción ([^;]+)').fillna('sin calefacción').astype('category')

# Extract year of construction
def extraer_ano_construccion(texto):
    match = re.search(r'construido en\s*(\d+)', texto.lower())
    return int(match.group(1)) if match else None

df['ano_construccion'] = df['Caracteristicas_basicas'].apply(lambda x: extraer_ano_construccion(str(x))).astype('Int64')

# Use regex to extract the number following the word "Planta"
df['planta_numero'] = df['Caracteristicas_basicas'].str.extract(r'Planta (\d+)')

# Convert the new column to a numeric type (integer)
df['planta_numero'] = pd.to_numeric(df['planta_numero'], errors='coerce').astype('Int64')

console.print("🏗️ [cyan]Año de construcción extraído correctamente.[/cyan]")

# Function to detect extra features
def extraer_caracteristicas_extras(texto):
    # Convert text to lowercase for standardization
    texto = texto.lower()
    
    # Detect presence of each feature
    jardin = 1 if 'jardín' in texto else 0
    piscina = 1 if 'piscina' in texto else 0
    aire_acondicionado = 1 if 'aire acondicionado' in texto else 0
    zonas_verdes = 1 if 'zonas verdes' in texto else 0
    
    return pd.Series([jardin, piscina, aire_acondicionado, zonas_verdes])

# Apply function to 'Caracteristicas_extra' and create new columns
df[['jardin', 'piscina', 'aire_acondicionado', 'zonas_verdes']] = df['Caracteristicas_extra'].apply(lambda x: extraer_caracteristicas_extras(str(x)))

# Drop original feature columns
df = df.drop(columns=['Caracteristicas_basicas', 'Caracteristicas_extra'], errors='ignore')

# Save processed DataFrame
output_path = '../data/casas_idealista_procesado_extra.csv'
df.to_csv(output_path, index=False, sep=';', encoding='utf-8')

console.print(f"💾 [green]Datos guardados en '{output_path}'.[/green]")

# Load files to combine
csv_file_1 = '../data/casas_idealista_procesado.csv'
csv_file_2 = '../data/casas_idealista_procesado_extra.csv'
output_csv_file = '../data/casas_idealista_procesado.csv'

def cargar_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8', sep=';')
        return df
    except FileNotFoundError:
        console.print(f"⚠️ [yellow]Archivo no encontrado: {file_path}[/yellow]")
        return pd.DataFrame()
    except Exception as e:
        console.print(f"❌ [red]Error al leer {file_path}: {e}[/red]")
        return pd.DataFrame()

df1 = cargar_csv(csv_file_1)
df2 = cargar_csv(csv_file_2)

# Combine DataFrames
if not df1.empty and not df2.empty:
    df_combined = pd.concat([df1, df2], ignore_index=True)
    console.print(f"📊 [cyan]Datos combinados: {len(df_combined)} registros.[/cyan]")
else:
    if df1.empty:
        console.print(f"⚠️ [yellow]El archivo '{csv_file_1}' está vacío.[/yellow]")
    if df2.empty:
        console.print(f"⚠️ [yellow]El archivo '{csv_file_2}' está vacío.[/yellow]")

# Save combined result
if not df_combined.empty:
    df_combined.to_csv(output_csv_file, index=False, sep=';', encoding='utf-8')
    console.print(f"✅ [green]Datos guardados en '{output_csv_file}'.[/green]")
else:
    console.print("❌ [red]No hay datos para guardar en el archivo final.[/red]")


# 3. Filtrado

In [ ]:
TEST_IDS = False  # Change to True to test sample IDs

# 📌 Define base path to save reports
base_path = "../reports"

# 📂 Load the file with new property IDs
console.print("[cyan]📂 Cargando datos desde 'casas_idealista_predicciones.csv'...[/cyan]")
df = pd.read_csv('../data/casas_idealista_predicciones.csv', sep=';')

# Convert IDs to integers and filter new properties
df["id"] = df["id"].astype(int)
if TEST_IDS:
    nuevos_ids = [107764183, 107763878]
else:
    nuevos_ids = [int(id) for id in nuevos_ids]
nuevas_casas = df[df["id"].isin(nuevos_ids)]

console.print(f"🏡 [yellow]Nuevos inmuebles detectados:[/yellow] {len(nuevas_casas)}")

# 📌 Apply threshold filter for weighted difference
console.print("📊 [cyan]Aplicando criterios de selección...[/cyan]")

# Define location groups
grupo_1 = [
    "Actur", "Centro", "Grancasa", "Almozara", "Pol Universidad Romareda",
    "Doctor Cerrada", "Universidad San Francisco", "Paseo Sagasta",
    "Paseo Independencia", "Casco Historico", "Alfonso", "Paseo de la Constitución"
]

grupo_2 = [
    "Ranillas", "Plaza de Toros", "Barrio del AVE", "Mercado San Valero"
]

clientes = pd.read_csv('../data/clientes.csv', sep=';')

# Calculate threshold for each property
def calcular_umbral(row):
    precio = row['Precio']
    barrio = row['Localizacion']
    umbral = 0
    if barrio in grupo_1:
        umbral = precio * 0.03 if 0 <= precio < 180000 else precio * 0.01 if 180000 <= precio <= 1500000 else 0
    elif barrio in grupo_2:
        umbral = precio * 0.05 if 0 <= precio < 150000 else precio * 0.03 if 150000 <= precio <= 1500000 else 0
    else:
        umbral = precio * 0.1 if 0 <= precio < 150000 else precio * 0.07 if 150000 <= precio <= 1500000 else 0
    return max(0, umbral)  # <- ensures it is not negative

nuevas_casas['Umbral'] = nuevas_casas.apply(calcular_umbral, axis=1)

# 📊 Apply filter by difference vs threshold
nuevas_casas = nuevas_casas[nuevas_casas["Diferencia_Ponderada"] >= nuevas_casas["Umbral"]].reset_index(drop=True)

# Create dictionary to map property index to list of interested clients
interesados_por_casa = {}

# Iterate over clients
for _, cliente in clientes.iterrows():
    # Convert values
    barrios = [b.strip() for b in str(cliente["barrios_interes"]).split(",") if b.strip()]
    estados = [e.strip().lower() for e in str(cliente["estados_vivienda"]).split(",") if e.strip()]
    extras = [e.strip() for e in str(cliente["extras"]).split(",") if e.strip()]

    filtro = pd.Series(True, index=nuevas_casas.index)

    # Price
    if pd.notna(cliente["precio_min"]):
        filtro &= nuevas_casas["Precio"] >= float(cliente["precio_min"])
    if pd.notna(cliente["precio_max"]):
        filtro &= nuevas_casas["Precio"] <= float(cliente["precio_max"])

    # Square meters
    if pd.notna(cliente["m2_min"]):
        filtro &= nuevas_casas["m2_utiles"] >= float(cliente["m2_min"])
    if pd.notna(cliente["m2_max"]):
        filtro &= nuevas_casas["m2_utiles"] <= float(cliente["m2_max"])

    # Floor
    if pd.notna(cliente["planta_min"]):
        filtro &= nuevas_casas["planta_numero"].fillna(-1000) >= float(cliente["planta_min"])
    if pd.notna(cliente["planta_max"]):
        filtro &= nuevas_casas["planta_numero"].fillna(1000) <= float(cliente["planta_max"])

    # Bedrooms
    if pd.notna(cliente["habitaciones_min"]):
        filtro &= nuevas_casas["habitaciones"] >= float(cliente["habitaciones_min"])

    # Bathrooms
    if pd.notna(cliente["baños_min"]):
        filtro &= nuevas_casas["banos"] >= float(cliente["baños_min"])

    # Exterior
    if pd.notna(cliente["exterior"]):
        filtro &= nuevas_casas["Exterior"].astype(str).str.lower().isin(["1", "true", "sí", "si"])

    # Neighborhoods (partial match)
    if barrios:
        filtro &= nuevas_casas["Localizacion"].str.lower().apply(
            lambda loc: any(barrio.lower() in loc for barrio in barrios)
        )

    # Property condition
    if estados:
        filtro &= nuevas_casas["estado_vivienda"].str.lower().isin(estados)

    # Preliminary filter applied
    candidatas = nuevas_casas[filtro].copy()

    # Extras (all desired must be active)
    for extra in extras:
        if extra in candidatas.columns:
            candidatas = candidatas[candidatas[extra].fillna(0).astype(int) == 1]

    # Associate client to each matching property
    for idx in candidatas.index:
        if idx not in interesados_por_casa:
            interesados_por_casa[idx] = []
        interesados_por_casa[idx].append(cliente['id'])

# Add column with list of interested clients
nuevas_casas['Clientes_Interesados'] = nuevas_casas.index.map(lambda idx: interesados_por_casa.get(idx, []))

# Filter properties with at least one interested client
notificaciones = nuevas_casas[nuevas_casas['Clientes_Interesados'].map(len) > 0].copy()

if not notificaciones.empty:
    console.print(f"✅ [green]{len(notificaciones)} inmuebles cumplen los criterios de inversión.[/green]")
else:
    console.print("⚠️ [red]No hay inmuebles interesantes para generar informes. Terminando ejecución.[/red]")
    exit()
    

# 📌 Generate CSV with filtered properties
notificaciones["Seleccionado"] = "No"
notificaciones["Enlace"] = "https://www.idealista.com/inmueble/" + notificaciones["id"].astype(str) + "/"
csv_path = "../csv/buenos.csv"

if os.path.exists(csv_path):
    os.remove(csv_path)
notificaciones.to_csv(csv_path, index=False, sep=";")

subprocess.run(["open", csv_path])  # Open file in Numbers

# 📌 Wait for CSV editing
console.print("🕒 [yellow]Esperando edición del archivo...[/yellow]")
last_modified = os.path.getmtime(csv_path)
while True:
    time.sleep(5)
    if os.path.getmtime(csv_path) != last_modified:
        console.print("✅ [green]Edición detectada. Continuando ejecución...[/green]")
        break

# 📌 Load edited CSV
notificaciones = pd.read_csv(csv_path, sep=";")
notificaciones["Seleccionado"] = notificaciones["Seleccionado"].astype(str).str.strip().str.lower()
notificaciones["Seleccionado"] = notificaciones["Seleccionado"].isin(["true", "1", "yes"])

# 📌 Filter only selected properties
notificaciones_marcadas = notificaciones[notificaciones["Seleccionado"] == True]

if not notificaciones_marcadas.empty:
    console.print(f"🚀 [green]Procesando {len(notificaciones_marcadas)} pisos seleccionados...[/green]")
else:
    console.print("⚠️ [red]No hay pisos seleccionados. Terminando ejecución.[/red]")
    exit()

# Function to start Selenium browser with anti-detection
def iniciar_navegador():
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    return uc.Chrome(
        options=options,
        use_subprocess=True,
        driver_executable_path="/Users/andres/chromedrivers/140/chromedriver"
    )

# Start browser
browser = iniciar_navegador()

info_dicts = {}

# 📌 Function to download image
def descargar_imagen(url, file_path):
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            console.print(f"✅ [green]Imagen guardada en {file_path}[/green]")
        else:
            console.print(f"⚠️ [yellow]No se pudo descargar la imagen desde {url}[/yellow]")
    except Exception as e:
        console.print(f"⚠️ [red]Error al descargar la imagen: {e}[/red]")

# List to store dictionaries for each property
listings_data = []

# 📌 Scraping and chart generation
for _, row in track(notificaciones_marcadas.iterrows(), total=len(notificaciones_marcadas), description="📊 Procesando informes..."):
    listing_id = str(row['id'])
    url_inmueble = f"https://www.idealista.com/inmueble/{listing_id}/"
    save_path = os.path.join(base_path, listing_id)
    os.makedirs(save_path, exist_ok=True)

    console.print(f"📂 [cyan]Guardando archivos en:[/cyan] {save_path}")

    listing = df[df['id'] == int(listing_id)]
    if listing.empty:
        console.print(f"⚠️ [red]El inmueble con ID {listing_id} no está en la base de datos.[/red]")
        continue

    # Convert row to dictionary
    listing_dict = listing.iloc[0].to_dict()

    # Scraping description and image
    try:
        browser.get(url_inmueble)
        time.sleep(random.uniform(4, 6))
        soup = bs(browser.page_source, 'lxml')

        # Get description
        descripcion = soup.find('div', {'class': 'adCommentsLanguage'})
        descripcion = descripcion.text.strip() if descripcion else "No disponible"

        console.print(f"📜 [cyan]Descripción del listing {listing_id}:[/cyan] {descripcion}")
        listing_dict["descripcion"] = descripcion

        # Get cover image (not included in dictionary)
        img_container = soup.find('picture', {'class': 'first-image'})
        img_tag = img_container.find('img') if img_container else None
        portada_url = img_tag['src'] if img_tag and 'src' in img_tag.attrs else None

        if portada_url:
            file_path_portada = os.path.join(save_path, f"Portada_{listing_id}.png")
            descargar_imagen(portada_url, file_path_portada)
        else:
            console.print(f"⚠️ [yellow]No se encontró imagen de portada para el listing {listing_id}[/yellow]")

        # Save dictionary in list
        listings_data.append(listing_dict)

    except Exception as e:
        console.print(f"⚠️ [red]Error al obtener descripción o imagen del listing {listing_id}: {e}[/red]")
        continue

browser.quit()
console.print("🏁 [bold green]Proceso completado.[/bold green] 🚀")


### 3.1 Informes

In [ ]:
# 📌 Function to clean text and remove accents/errors
def limpiar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(c)
    )

# 📌 Load LaTeX template from file
with open("../reports/plantilla_informe.tex", "r", encoding="utf-8") as f:
    plantilla = f.read()

# 📌 Function to remove image block if cover file does not exist
def ajustar_latex_si_no_hay_portada(tex_code, listing_id, save_path):
    portada_path = os.path.join(save_path, f"Portada_{listing_id}.png")
    if not os.path.exists(portada_path):
        # Remove block between \begin{tcolorbox} and \end{tcolorbox}
        inicio = tex_code.find(r"\begin{tcolorbox}")
        fin = tex_code.find(r"\end{tcolorbox}") + len(r"\end{tcolorbox}")
        if inicio != -1 and fin != -1 and fin > inicio:
            tex_code = tex_code[:inicio] + tex_code[fin:]
    return tex_code

def generar_tex(info):
    estimacion = round((info["Precio"] + info["Diferencia_Ponderada"]) / 10000) * 10000
    minimo = max(
        int(10000 * ((info["Precio"] + info["Diferencia_Ponderada"] - 10000 - 0.66 * info["Diferencia_Ponderada"]) // 10000)),
        int(((info["Precio"] + 9999) // 10000) * 10000)
    )
    maximo = int(10000 * ((info["Precio"] + info["Diferencia_Ponderada"] + 10000 - 0.66 * info["Diferencia_Ponderada"]) // 10000))

    # 📌 Generate interval as formatted LaTeX string
    if minimo == maximo:
        intervalo = f"\\num{{{minimo}}} €"
    else:
        intervalo = f"\\num{{{minimo}}} - \\num{{{maximo}}} €"
    
    valores = {
        "ID": info["id"],
        "LOCALIZACION": info["Localizacion"],
        "PRECIO": int(info["Precio"]),
        "M2_CONSTRUIDOS": int(info["m2_construidos"]),
        "M2_UTIL": int(info["m2_utiles"]),
        "HABITACIONES": int(info["habitaciones"]),
        "BANOS": int(info["banos"]),
        "ESTADO": info["estado_vivienda"],
        "ASCENSOR": "Sí" if info["ascensor"] else "No",
        "CALEFACCION": info["Calefacción"],
        "ANO": int(info["ano_construccion"]),
        "ESTIMACION": estimacion,
        "MINIMO": minimo,
        "MAXIMO": maximo,
        "INTERVALO": intervalo,
    }

    tex = plantilla
    for clave, valor in valores.items():
        tex = tex.replace(f"{{{{{clave}}}}}", str(valor))
    return tex

# 📌 Check if there are interesting properties
if notificaciones_marcadas.empty:
    console.print("⚠️ [red]No hay nuevos inmuebles interesantes para generar informes. Terminando ejecución.[/red]")
    exit()

# 📌 Define base path to save reports
base_path = "../reports"
os.makedirs(base_path, exist_ok=True)

# 📌 List to store generated reports
informes_generados = []

# 🔹 Generate LaTeX reports for selected properties
for i, (_, row) in enumerate(notificaciones_marcadas.iterrows()):
    listing_id = str(row['id'])
    save_path = os.path.join(base_path, listing_id)
    os.makedirs(save_path, exist_ok=True)  # Ensure folder exists

    file_path_tex = os.path.join(save_path, f"informe_{listing_id}.tex")
    file_path_pdf = os.path.join(save_path, f"informe_{listing_id}.pdf")

    # 🔹 Get corresponding dictionary WITHOUT converting to JSON yet
    info_df = listings_data[i]

    # ✅ Generate LaTeX code automatically without OpenAI
    tex_code = generar_tex(info_df)

    # 🔧 Remove image block if cover does not exist
    tex_code = ajustar_latex_si_no_hay_portada(tex_code, listing_id, save_path)

    # 🔹 Save LaTeX code into `.tex` file
    with open(file_path_tex, "w", encoding="utf-8") as f:
        f.write(tex_code)

    # 🔹 Compile LaTeX to PDF with `pdflatex`
    try:
        subprocess.run(
            ["pdflatex", "-output-directory", save_path, file_path_tex],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        console.print(f"✅ PDF generado con éxito: {file_path_pdf}")
        informes_generados.append(file_path_pdf)
    except subprocess.CalledProcessError as e:
        console.print(f"❌ Error al compilar LaTeX para {listing_id}.")
        console.print(f"🔍 Detalles del error:\n{e.stderr.decode()}")

console.print("🏁 [bold green]Proceso de generación de informes completado.[/bold green] 🚀")   

### 3.2 LaTeX y envio

In [ ]:
from ast import literal_eval

# 📌 Check if there are generated reports
if not informes_generados:
    console.print("⚠️ [red]No hay informes generados. No se enviará correo.[/red]")
    exit()

# ✅ Convert string column to real lists
notificaciones['Clientes_Interesados'] = notificaciones['Clientes_Interesados'].apply(
    lambda x: x if isinstance(x, list) else literal_eval(x)
)

# 📦 Build dictionary of reports by client
informes_por_cliente = defaultdict(list)

# Iterate over generated reports and match with `notificaciones`
for file_path_pdf in informes_generados:
    filename = os.path.basename(file_path_pdf)  # Ex: "informe_123456.pdf"
    listing_id = int(filename.replace("informe_", "").replace(".pdf", ""))
    
    fila = notificaciones[notificaciones['id'] == listing_id]

    if not fila.empty:
        interesados = fila.iloc[0]['Clientes_Interesados']
        for cliente_id in interesados:
            informes_por_cliente[str(cliente_id)].append(file_path_pdf)  # keys as str

# ✉️ Send emails by client
for _, cliente in clientes.iterrows():
    cliente_id = str(cliente['id'])  # key as str
    email_cliente = cliente['email']
    informes_cliente = informes_por_cliente.get(cliente_id, [])

    if not informes_cliente:
        continue  # no reports for this client

    msg = EmailMessage()
    msg["Subject"] = "Informes inmobiliarios recientes - Oportunidades destacadas para ti"
    msg["From"] = EMAIL_SENDER
    msg["To"] = email_cliente
    msg.set_content(
        f"""
        Estimado/a {cliente['nombre']},

        Te adjuntamos los informes más recientes que cumplimentan tus preferencias de inversión. 

        Un cordial saludo,  
        El equipo de Inmobil-IA-ria.
        """
    )

    for file_path_pdf in informes_cliente:
        try:
            with open(file_path_pdf, "rb") as f:
                filename = os.path.basename(file_path_pdf)
                msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=filename)
            console.print(f"📎 Informe {filename} adjuntado para {email_cliente}")
        except Exception as e:
            console.print(f"❌ Error al adjuntar {filename} para {email_cliente}: {e}")

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        console.print(f"✅ Correo enviado a {email_cliente}")
    except smtplib.SMTPAuthenticationError:
        console.print(f"❌ Error de autenticación al enviar a {email_cliente}")
    except smtplib.SMTPException as e:
        console.print(f"❌ Error al enviar a {email_cliente}: {e}")